In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model as skl
import pandas as pd
%matplotlib inline

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report 
from sklearn.ensemble import RandomForestClassifier

## Tuning the hyper-parameters of your estimator
- Train models with different combinations of hyper parameters
- Use cross validation to evaluate the performance of models
- Choose the best one


In [3]:
# loading training/testing data
all_data = pd.read_csv("preprocessed_data.csv")
X = all_data.drop(["Survived"], axis=1)
y = all_data["Survived"]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1)
print(len(X_train))
print(len(X_test))

712
179


In [5]:
model = RandomForestClassifier(max_depth=100, n_estimators=10)
model.fit(X_train, y_train)
predict_y = model.predict(X_train)
print("training accuracy:", (predict_y == y_train).sum()/ len(predict_y))


predict_y = model.predict(X_test)
print("testing accuracy:", (predict_y == y_test).sum()/ len(predict_y))

training accuracy: 0.976123595505618
testing accuracy: 0.7653631284916201


In [6]:
cross_val_score(model, X, y, cv=5, scoring="accuracy")

array([0.79329609, 0.81460674, 0.82022472, 0.76404494, 0.80898876])

You can see more classification metics here: https://scikit-learn.org/stable/modules/model_evaluation.html

In [7]:
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 100,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [8]:
# Set the parameters by cross-validation
tuned_parameters = [{'n_estimators': [5,10,30,50,100], 'max_depth': [3,5,10,20]}]


clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=5,
                   scoring='accuracy')
clf.fit(X_train, y_train)

print("best parameters: ", clf.best_params_)
print("best score:", clf.best_score_ )
print(clf.cv_results_['mean_test_score'])
print(clf.cv_results_['std_test_score'])
print(clf.cv_results_["params"])

best parameters:  {'max_depth': 10, 'n_estimators': 100}
best score: 0.8399684822220033
[0.77815424 0.81468531 0.81330641 0.80486556 0.82030927 0.82172757
 0.81607407 0.83014872 0.83298532 0.82733182 0.82452477 0.82312617
 0.83437408 0.81610361 0.83996848 0.79637546 0.80907121 0.81608392
 0.81890082 0.82591352]
[0.03777061 0.0311109  0.03305746 0.0315592  0.03108962 0.02969403
 0.02947055 0.02866537 0.03923541 0.03202937 0.02748529 0.03220612
 0.03285663 0.02932389 0.03011251 0.01125347 0.03766947 0.03291348
 0.02835808 0.02809083]
[{'max_depth': 3, 'n_estimators': 5}, {'max_depth': 3, 'n_estimators': 10}, {'max_depth': 3, 'n_estimators': 30}, {'max_depth': 3, 'n_estimators': 50}, {'max_depth': 3, 'n_estimators': 100}, {'max_depth': 5, 'n_estimators': 5}, {'max_depth': 5, 'n_estimators': 10}, {'max_depth': 5, 'n_estimators': 30}, {'max_depth': 5, 'n_estimators': 50}, {'max_depth': 5, 'n_estimators': 100}, {'max_depth': 10, 'n_estimators': 5}, {'max_depth': 10, 'n_estimators': 10}, {'ma

In [9]:
model = RandomForestClassifier(max_depth=5, n_estimators=30)
cross_val_score(model, X, y, cv=5, scoring="accuracy")

array([0.81564246, 0.8258427 , 0.82022472, 0.79775281, 0.83146067])

In [10]:
print("Detailed classification report:") 
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

Detailed classification report:
              precision    recall  f1-score   support

           0       0.75      0.90      0.82       106
           1       0.79      0.58      0.67        73

    accuracy                           0.77       179
   macro avg       0.77      0.74      0.74       179
weighted avg       0.77      0.77      0.76       179


